### In this file I will produce the data for feature selection 3, explain reasons for design choices<br>
specifically I make separate pickles for doubled data and single data, remove low variance and highly correlated features and fingerprints<br>
also I study reasons for data loss

In [1]:
import numpy as np
import pandas as pd
import _pickle as pickle
from rdkit import Chem
from rdkit.Chem import MACCSkeys

In [3]:
with open('./pickles/doubledData.pkl', 'rb') as inp:
    dataDoubled = pickle.load(inp)

with open('./pickles/singleData.pkl', 'rb') as inp:
    dataSingle = pickle.load(inp)

with open('./pickles/joinFrame.pkl', 'rb') as inp:
    drugs = pickle.load(inp)

In [ ]:
# research for later: Figure out why
data[(data["Drug1"]=="mk-8776") & (data["Drug2"]=="bez-235")].head(2)

### Preparation of Fingerprint - Data

In [2]:
# getting chemical descriptors
chemDescr = pd.read_csv("../drugCombDBdata/descriptions/drug_chemical_info.csv")

#removing empty values
print(str(len(chemDescr[chemDescr["smilesString"]=="none"]))+" drugs with 'none' as smile")
print(str(len(chemDescr[pd.isna(chemDescr["smilesString"])]))+" drugs with NaN as smile")
print("     dropping both for now")


chemDescr = chemDescr[chemDescr["smilesString"]!="none"]
chemDescr = chemDescr[~pd.isna(chemDescr["smilesString"])]
chemDescr.reset_index(inplace=True)
chemDescr.head(2)

30 drugs with 'none' as smile
454 drugs with NaN as smile
     dropping both for now


,index,drugName,cIds,drugNameOfficial,molecularWeight,smilesString
0,0,Bendamustine,CIDs00065628,bendamustine,358.26284,CN1C2=C(C=C(C=C2)N(CCCl)CCCl)N=C1CCCC(=O)O
1,1,Lonidamine,CIDs00039562,lonidamine,321.15810,C1=CC=C2C(=C1)C(=NN2CC3=C(C=C(C=C3)Cl)Cl)C(=O)O


In [15]:
chemDescr["mol-object"]=[Chem.MolFromSmiles(mol) for mol in chemDescr["smilesString"]]

maccsFrame = pd.DataFrame(chemDescr["drugName"])

maccsFrame["MACCSk"] = [np.array(Chem.MACCSkeys.GenMACCSKeys(mol)) for mol in chemDescr["mol-object"]]
maccsFrame.head(2)

,drugName,MACCSk
0,Bendamustine,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,Lonidamine,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [17]:
joinMaccs = pd.DataFrame([])
for i in range(len(maccsFrame)):
    joinMaccs = joinMaccs.append(pd.Series(maccsFrame.iloc[i,1]),ignore_index=True)
joinMaccs.head(2)

,0,1,2,3,4,5,6,7,8,9,...,157,158,159,160,161,162,163,164,165,166
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0


In [18]:
drugMACCS = maccsFrame.join(joinMaccs,how="left").drop(columns="MACCSk")
with open('./pickles/drugMACCS.pkl', 'wb') as outp:
    pickle.dump(drugMACCS, outp)
drugMACCS.head(2)

,drugName,0,1,2,3,4,5,6,7,8,...,157,158,159,160,161,162,163,164,165,166
0,Bendamustine,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0
1,Lonidamine,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0


### Studying the problem in the data

In [19]:
#replicate demoonstration-ammar, showing weak to edit data and especially whats wrong with the found patterns

sklearn random forrest can be used for feat selection too - feature_importances_ <br><br>

remove low variance features<br>
remove highly correlated features too<br>
finish the demonstration file<br>
look at the distribution of the y-variable to do balanced splitting "stratified split" in sklearn<br>
(all of that is already implemented)<br><br>

try fingerprints splitting over cols <br>
study causes of data loss <br>
look at synapse paper as well<br>
compare performance of HSA to other properties<br>
PRETTYFY - make notes for experiements, explain what does what and why